In [1]:
import torch
from PIL import Image
import numpy as np
import open_clip

# model, _, preprocess = open_clip.create_model_and_transforms('ViT-bigG-14', pretrained='laion2b_s39b_b160k')
pretrained_model = '/mnt/eds_share/Users/yilu.zhou/Development/log/open_clip/2023_07_17-13_09_01-model_ViT-L-14-lr_1e-06-b_256-j_4-p_amp/checkpoints/epoch_30.pt'
model, _, preprocess = open_clip.create_model_and_transforms('ViT-L-14', pretrained=pretrained_model)
tokenizer = open_clip.get_tokenizer('ViT-L-14')

In [2]:
# open_clip.list_pretrained()
import numpy as np
model.eval()
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Context length:", context_length)
print("Vocab size:", vocab_size)

Model parameters: 427,616,513
Context length: 77
Vocab size: 49408


In [3]:
image_path = '/mnt/eds_ml/Users/Yilu_ML/roco-dataset/data/test/radiology/images/ROCO_00001.jpg'
image = preprocess(Image.open(image_path)).unsqueeze(0)
text = tokenizer(["MRI", "CT"])

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)  # prints: [[1., 0., 0.]]

Label probs: tensor([[0.9975, 0.0025]])


In [14]:
import pandas as pd
df = pd.read_csv('./data/roco_validation.csv', sep='\t')
img_key = 'filepath'
print(df.keys())
images = df[img_key].tolist()
print(images[:5])
caption_key = 'title'
captions = df[caption_key].tolist()

Index(['filepath', 'caption'], dtype='object')
['E:\\Work\\roco-dataset\\data\\validation\\radiology\\images\\ROCO_00020.jpg', 'E:\\Work\\roco-dataset\\data\\validation\\radiology\\images\\ROCO_00027.jpg', 'E:\\Work\\roco-dataset\\data\\validation\\radiology\\images\\ROCO_00059.jpg', 'E:\\Work\\roco-dataset\\data\\validation\\radiology\\images\\ROCO_00062.jpg', 'E:\\Work\\roco-dataset\\data\\validation\\radiology\\images\\ROCO_00068.jpg']


In [4]:
import open_clip
open_clip.list_pretrained()

[('RN50', 'openai'),
 ('RN50', 'yfcc15m'),
 ('RN50', 'cc12m'),
 ('RN50-quickgelu', 'openai'),
 ('RN50-quickgelu', 'yfcc15m'),
 ('RN50-quickgelu', 'cc12m'),
 ('RN101', 'openai'),
 ('RN101', 'yfcc15m'),
 ('RN101-quickgelu', 'openai'),
 ('RN101-quickgelu', 'yfcc15m'),
 ('RN50x4', 'openai'),
 ('RN50x16', 'openai'),
 ('RN50x64', 'openai'),
 ('ViT-B-32', 'openai'),
 ('ViT-B-32', 'laion400m_e31'),
 ('ViT-B-32', 'laion400m_e32'),
 ('ViT-B-32', 'laion2b_e16'),
 ('ViT-B-32', 'laion2b_s34b_b79k'),
 ('ViT-B-32', 'datacomp_m_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_clip_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_laion_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_image_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_text_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_basic_s128m_b4k'),
 ('ViT-B-32', 'commonpool_m_s128m_b4k'),
 ('ViT-B-32', 'datacomp_s_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_clip_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_laion_s13m_b4k'),
 ('ViT-B-32', 'commonpool_s_image_s13m_b4k'),
 ('ViT-B-32', 'commo

In [9]:
import open_clip
import torch
from PIL import Image
#('coca_ViT-B-32', 'laion2b_s13b_b90k')
# model, _, transform = open_clip.create_model_and_transforms(
#   model_name="coca_ViT-B-32",  #coca_ViT-L-14, 
#   pretrained="laion2b_s13b_b90k", #mscoco_finetuned_laion2B-s13B-b90k
# )
# /mnt/eds_share/Users/yilu.zhou/Development/log/open_clip/2023_07_28-13_55_44-model_coca_ViT-L-14-lr_1e-05-b_32-j_4-p_amp/checkpoints/epoch_1.pt
model, _, transform = open_clip.create_model_and_transforms(
  model_name="coca_ViT-L-14",
  pretrained="mscoco_finetuned_laion2B-s13B-b90k"
)

image_path = '/mnt/eds_ml/Users/Yilu_ML/roco-dataset/data/test/radiology/images/ROCO_00001.jpg'
im = Image.open(image_path).convert("RGB")
im = transform(im).unsqueeze(0)

with torch.no_grad(), torch.cuda.amp.autocast():
  generated = model.generate(im)

print(open_clip.decode(generated[0]).split("<end_of_text>")[0].replace("<start_of_text>", ""))

a picture of the human head in a mri . 


In [12]:
import torch
from PIL import Image
import numpy as np
import open_clip

# /mnt/eds_share/Users/yilu.zhou/Development/log/open_clip/2023_07_28-13_55_44-model_coca_ViT-L-14-lr_1e-05-b_32-j_4-p_amp/checkpoints/epoch_3.pt
# model, _, preprocess = open_clip.create_model_and_transforms('coca_ViT-L-14', pretrained='mscoco_finetuned_laion2B-s13B-b90k')
model, _, preprocess = open_clip.create_model_and_transforms(
  model_name="coca_ViT-L-14",
  pretrained="mscoco_finetuned_laion2B-s13B-b90k"
)

tokenizer = open_clip.get_tokenizer('coca_ViT-L-14')

image_path = '/mnt/eds_ml/Users/Yilu_ML/roco-dataset/data/test/radiology/images/ROCO_00001.jpg'
image = preprocess(Image.open(image_path)).unsqueeze(0)
text = tokenizer(["MRI", "CT", "Xray"])

with torch.no_grad(), torch.cuda.amp.autocast():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)

print("Label probs:", text_probs)  # prints: [[1., 0., 0.]]

Label probs: tensor([[9.9364e-01, 3.2589e-04, 6.0292e-03]])


In [ ]:
# Direct Classification benchmark
import torch, os, open_clip
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm

pretrained_model = 'mscoco_finetuned_laion2B-s13B-b90k'
sentences = ["key1", "key2"]
img_dir = 'img_dir'
model, _, preprocess = open_clip.create_model_and_transforms(
    model_name="coca_ViT-L-14",
    pretrained=pretrained_model,
)
tokenizer = open_clip.get_tokenizer('coca_ViT-L-14')

df_all = pd.DataFrame()

# Prepare the tokenizer for sentences
text = tokenizer(sentences)
with torch.no_grad(), torch.cuda.amp.autocast():
    text_features = model.encode_text(text)
    text_features /= text_features.norm(dim=-1, keepdim=True)


def read_single_image(img_dir, img_path, sentences, text_features):
    #get the ground truth value
    # Extract the filename without extension
    filename = os.path.splitext(os.path.basename(img_path))[0]
    image_path = os.path.join(img_dir, img_path)
    # Check if any of the words in sentences are in the filename
    ground_truth = [word for word in sentences if word in filename][0]
    
    # load a sample image
    image = preprocess(Image.open(image_path)).unsqueeze(0)
    with torch.no_grad(), torch.cuda.amp.autocast():
        image_features = model.encode_image(image)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
        text_probs = text_probs.cpu().tolist()[0]

    # Construct the dictionary
    class_predict_dict = dict(zip(sentences, text_probs))
    # Extract the key with the largest value
    class_predict = max(class_predict_dict, key=class_predict_dict.get)
    
    if ground_truth == class_predict:
        predicted_correct = 1
    else:
        predicted_correct = 0
    
    pred_dict = {'img_path': [img_path], 'class_predict_dict': [class_predict_dict], 'ground_truth': [ground_truth], 'class_predict': [class_predict], 'predicted_correct': [predicted_correct]}
    df_single = pd.DataFrame(pred_dict)
    
    return df_single


all_images = os.listdir(img_dir)
for img_path in tqdm(all_images, desc="Processing images", unit="image"):
    if '_full_' in img_path: #filaname
        continue #skip full images
    df = read_single_image(img_dir, img_path, sentences, text_features)
    df_all = pd.concat([df_all, df]).reset_index(drop=True)

# save csv file
sentences_str = "_".join(sentences)
df_all.to_csv(f"{sentences_str}.csv", index=False)
# Count the number of rows in the 'predicted_correct' column
total_rows = len(df_all)
# Count the occurrences of '1' in the 'predicted_correct' column
count_ones = df_all['predicted_correct'].sum()
print(f"Total images: {total_rows}")
print(f"predicted_correct': {count_ones}")
predicted_correct_pct = count_ones /total_rows *100
print(f"predicted_correct (%)': {predicted_correct_pct}%")

In [ ]:
# Only 1 bounding box
import torch
from PIL import Image, ImageEnhance, ImageDraw, ImageColor
import numpy as np
import open_clip
import matplotlib.pyplot as plt 


model, _, preprocess = open_clip.create_model_and_transforms(
  model_name="coca_ViT-L-14",
  pretrained="mscoco_finetuned_laion2B-s13B-b90k"
)

tokenizer = open_clip.get_tokenizer('coca_ViT-L-14')

image_path = 'Your image path'
img = Image.open(image_path).convert('RGB')
text = tokenizer(["keyword1", "keyword2"]) 

# Adapted from https://github.com/openai/CLIP/issues/82
def generate_crop_dict(img, preprocess):
    # Convert image into 56 by 56 crops
    offset = 56
    image_crop_dict = {}
    for h in range(0, 224, offset):
        for w in range(0, 224, offset):
            area = (w, h, w + offset, h + offset)
            crop = img.crop(area)
            crop = crop.resize((224, 224), Image.Resampling.LANCZOS)
            # Include the batch dimension
            image_crop_dict[area] = torch.tensor(preprocess(crop)).unsqueeze(0)
    return image_crop_dict


def highlight_area(img, region, factor, outline_color=None, outline_width=1):
    """ Highlight specified rectangular region of image by `factor` with an
        optional colored  boarder drawn around its edges and return the result.
    """
    img = img.copy()  # Avoid changing original image.
    img_crop = img.crop(region)

    # brightner = ImageEnhance.Brightness(img_crop)
    # img_crop = brightner.enhance(factor)
    img.paste(img_crop, region)
    # Optionally draw a colored outline around the edge of the rectangular region.
    if outline_color:
        draw = ImageDraw.Draw(img)  # Create a drawing context.
        left, upper, right, lower = region  # Get bounds.
        coords = [(left, upper), (right, upper), (right, lower), (left, lower),
                  (left, upper)]
        draw.line(coords, fill=outline_color, width=outline_width)
    return img


def viz_pseudo_attn_maps(model, img, text, preprocess):
    image_dict = generate_crop_dict(img, preprocess)
    sim_dict = {}
    for crop_loc, image in image_dict.items():
        with torch.no_grad():
            image_features = model.encode_image(image).float()
            text_features = model.encode_text(text).float()
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)
            similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T
            # print(f'similarity = {similarity}')
            sim_dict[crop_loc] = similarity #.item()
    print(f'sim_dict = {sim_dict}')
    # bbox = max(sim_dict, key=sim_dict.get)
    # bbox = max(sim_dict, key=lambda k: sim_dict[k][0])
    bbox = max(sim_dict, key=lambda k: sim_dict[k][0])
    print(f"Max patch is {bbox}")
    # We pass the original image and the rectangle.
    red = ImageColor.getrgb('red')
    img2 = highlight_area(img, bbox, 2.5, outline_color=red, outline_width=2)
    plt.imshow(img2)
    plt.show()
    return sim_dict

sim_dict = viz_pseudo_attn_maps(model, img, text, preprocess)

In [ ]:
# Generating heat map of attention mask
import torch, os
from PIL import Image, ImageEnhance, ImageDraw, ImageColor
import numpy as np
import open_clip
import matplotlib.pyplot as plt 

model, _, preprocess = open_clip.create_model_and_transforms(
  model_name="coca_ViT-L-14",
  pretrained="mscoco_finetuned_laion2B-s13B-b90k"
)

tokenizer = open_clip.get_tokenizer('coca_ViT-L-14')
image_path = 'IMAGE path'
sentences = ["keywords1", "keywords2"]
filename = os.path.splitext(os.path.basename(image_path))[0]
print(f'image name: {filename}')
img = Image.open(image_path).convert('RGB')
text = tokenizer(sentences)


# https://github.com/openai/CLIP/issues/82
def generate_crop_dict(img, preprocess):
    # Convert image into 56 by 56 crops
    shortest_pixel = min(img.size)
    offset = min(56, int(shortest_pixel/8))
    image_crop_dict = {}
    for h in range(0, 224, offset):
        for w in range(0, 224, offset):
            area = (w, h, w + offset, h + offset)
            crop = img.crop(area)
            crop = crop.resize((224, 224), Image.Resampling.LANCZOS)
            # Include the batch dimension
            # image_crop_dict[area] = torch.tensor(preprocess(crop)).unsqueeze(0)
            image_crop_dict[area] = preprocess(crop).clone().detach().unsqueeze(0)
    return image_crop_dict


def highlight_area(img, region, factor, outline_color=None, outline_width=1):
    """ Highlight specified rectangular region of image by `factor` with an
        optional colored  boarder drawn around its edges and return the result.
    """
    img = img.copy()  # Avoid changing original image.
    img_crop = img.crop(region)

    # brightner = ImageEnhance.Brightness(img_crop)
    # img_crop = brightner.enhance(factor)
    img.paste(img_crop, region)
    # Optionally draw a colored outline around the edge of the rectangular region.
    if outline_color:
        draw = ImageDraw.Draw(img)  # Create a drawing context.
        left, upper, right, lower = region  # Get bounds.
        coords = [(left, upper), (right, upper), (right, lower), (left, lower),
                  (left, upper)]
        draw.line(coords, fill=outline_color, width=outline_width)
    return img


def viz_pseudo_attn_maps(model, img, text, preprocess, sentences):
    image_dict = generate_crop_dict(img, preprocess)
    sim_dict = {}
    for crop_loc, image in image_dict.items():
        with torch.no_grad():
            image_features = model.encode_image(image).float()
            text_features = model.encode_text(text).float()
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)
            similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T
            # print(f'similarity = {similarity}')
            sim_dict[crop_loc] = similarity #.item()

    # print(f'sim_dict={sim_dict}')
    # load a sample image
    image = preprocess(img).unsqueeze(0)
    with torch.no_grad(), torch.cuda.amp.autocast():
        image_features = model.encode_image(image)
        image_features /= image_features.norm(dim=-1, keepdim=True)
        text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
        text_probs = text_probs.cpu().tolist()[0]
    #extract the prediction            
    text_probs = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    text_probs = text_probs.cpu().tolist()[0]

    # Construct the dictionary
    class_predict_dict = dict(zip(sentences, text_probs))
    # Extract the key with the largest value
    predicted_class = max(class_predict_dict, key=class_predict_dict.get)
    print(f'The heatmap of predicted_class ({predicted_class}) is as shown below:')
    if predicted_class in sentences:
        pred_choice = sentences.index(predicted_class)
    else:
        print("Predicted class not found in sentences.")


    # pred_choice = 0
    # print(f'sim_dict={sim_dict}')
    bbox = max(sim_dict, key=lambda k: sim_dict[k][pred_choice,0])
    # print(f"Max spot is {bbox}")
    # Pass the original image and the rectangle.
    red = ImageColor.getrgb('red')
    img2 = highlight_area(img, bbox, 2.5, outline_color=red, outline_width=2)
    
    # Create a 2D array with the same shape as your image (divided by the crop size)
    heatmap = np.zeros((img.size[1], img.size[0]))
    for crop_loc, similarity in sim_dict.items():
        heatmap[crop_loc[1]:crop_loc[3], crop_loc[0]:crop_loc[2]] = similarity[pred_choice, 0]
    # Normalize the heatmap
    heatmap = (heatmap - np.min(heatmap)) / (np.max(heatmap) - np.min(heatmap))
    
    # Plot original image with bounding box and heatmap side by side
    # Create an additional subplot for the colorbar
    fig, axs = plt.subplots(1, 3, figsize=(11, 5), gridspec_kw={'width_ratios': [1, 1, 0.05]})
    # fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(img2)
    axs[0].axis('on')
    im = axs[1].imshow(heatmap, cmap='Blues')
    # axs[1].imshow(heatmap, cmap='Blues', interpolation='nearest')
    # axs[1].colorbar()
    axs[1].set_xticks([])
    axs[1].set_yticks([])
    axs[1].set_frame_on(True)
    # Add a colorbar to the heatmap
    fig.colorbar(im, cax=axs[2]) #, orientation='vertical'
    axs[1].axis('on')
    plt.show()
    return sim_dict

sim_dict = viz_pseudo_attn_maps(model, img, text, preprocess, sentences)

In [ ]:
# Generating heat map of attention mask for caption prediction
import open_clip, torch, os
from PIL import Image, ImageEnhance, ImageDraw, ImageColor
import numpy as np
import open_clip
import matplotlib.pyplot as plt

model, _, preprocess = open_clip.create_model_and_transforms(
    model_name="coca_ViT-L-14",
    pretrained=""
)
tokenizer = open_clip.get_tokenizer('coca_ViT-L-14')

image_path = ''
img = Image.open(image_path).convert("RGB")
img = img.resize((224, 224), Image.Resampling.LANCZOS)
im = preprocess(img).unsqueeze(0)

with torch.no_grad(), torch.cuda.amp.autocast():
    generated = model.generate(im)

# Generating heat map of attention mask for sentences
sentences = [open_clip.decode(generated[0]).split("<end_of_text>")[0].replace("<start_of_text>", "")]

filename = os.path.splitext(os.path.basename(image_path))[0]
print(f'image_path: {image_path}')
print(f'image name: {filename}.jpg \nPredicted caption: {sentences[0]}')
print(f'The heatmap for predicted caption shown as below:')
text = tokenizer(sentences)


# https://github.com/openai/CLIP/issues/82
def generate_crop_dict(img, preprocess):
    # Convert image into 56 by 56 crops
    shortest_pixel = min(img.size)
    offset = min(56, int(shortest_pixel/8))
    image_crop_dict = {}
    for h in range(0, 224, offset):
        for w in range(0, 224, offset):
            area = (w, h, w + offset, h + offset)
            crop = img.crop(area)
            crop = crop.resize((224, 224), Image.Resampling.LANCZOS)
            # Include the batch dimension
            image_crop_dict[area] = preprocess(crop).clone().detach().unsqueeze(0)
    return image_crop_dict


def highlight_area(img, region, factor, outline_color=None, outline_width=1):
    """ Highlight specified rectangular region of image by `factor` with an
        optional colored  boarder drawn around its edges and return the result.
    """
    img = img.copy()  # Avoid changing original image.
    img_crop = img.crop(region)

    # brightner = ImageEnhance.Brightness(img_crop)
    # img_crop = brightner.enhance(factor)
    img.paste(img_crop, region)
    # Optionally draw a colored outline around the edge of the rectangular region.
    if outline_color:
        draw = ImageDraw.Draw(img)  # Create a drawing context.
        left, upper, right, lower = region  # Get bounds.
        coords = [(left, upper), (right, upper), (right, lower), (left, lower),
                  (left, upper)]
        draw.line(coords, fill=outline_color, width=outline_width)
    return img


def viz_pseudo_attn_maps(model, img, text, preprocess, sentences):
    image_dict = generate_crop_dict(img, preprocess)
    sim_dict = {}
    for crop_loc, image in image_dict.items():
        with torch.no_grad():
            image_features = model.encode_image(image).float()
            text_features = model.encode_text(text).float()
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)
            similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T
            # print(f'similarity = {similarity}')
            sim_dict[crop_loc] = similarity #.item()

    # print(f'sim_dict={sim_dict}')
    pred_choice = 0
    bbox = max(sim_dict, key=lambda k: sim_dict[k][pred_choice,0])
    # print(f"Max spot is {bbox}")
    # Pass the original image and the rectangle.
    red = ImageColor.getrgb('red')
    img2 = highlight_area(img, bbox, 2.5, outline_color=red, outline_width=2)
    
    # Create a 2D array with the same shape as your image (divided by the crop size)
    heatmap = np.zeros((img.size[1], img.size[0]))
    for crop_loc, similarity in sim_dict.items():
        heatmap[crop_loc[1]:crop_loc[3], crop_loc[0]:crop_loc[2]] = similarity[pred_choice, 0]
    # Normalize the heatmap
    heatmap = (heatmap - np.min(heatmap)) / (np.max(heatmap) - np.min(heatmap))
    
    # Plot original image with bounding box and heatmap side by side
    # Create an additional subplot for the colorbar
    fig, axs = plt.subplots(1, 3, figsize=(11, 5), gridspec_kw={'width_ratios': [1, 1, 0.05]})
    # fig, axs = plt.subplots(1, 2, figsize=(10, 5))
    axs[0].imshow(img2)
    axs[0].axis('on')
    im = axs[1].imshow(heatmap, cmap='Blues')
    # axs[1].imshow(heatmap, cmap='Blues', interpolation='nearest')
    # axs[1].colorbar()
    axs[1].set_xticks([])
    axs[1].set_yticks([])
    axs[1].set_frame_on(True)
    # Add a colorbar to the heatmap
    fig.colorbar(im, cax=axs[2]) #, orientation='vertical'
    axs[1].axis('on')
    plt.show()
    return sim_dict

sim_dict = viz_pseudo_attn_maps(model, img, text, preprocess, sentences)

In [ ]:
# Indirect Classification benchmark (first convert to caption, then do classfication)
import torch, os, open_clip
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm

pretrained_model=""
sentences = ["key1", "key2"]
img_dir = ''
model, _, preprocess = open_clip.create_model_and_transforms(
    model_name="coca_ViT-L-14",
    pretrained=pretrained_model,
)
tokenizer = open_clip.get_tokenizer('coca_ViT-L-14')
df_all = pd.DataFrame()

def read_single_image(img_dir, img_path, sentences):
    #get the ground truth value
    # Extract the filename without extension
    filename = os.path.splitext(os.path.basename(img_path))[0]
    image_path = os.path.join(img_dir, img_path)
    # Check if any of the words in sentences are in the filename
    ground_truth = [word for word in sentences if word in filename][0]
    
    # load a sample image
    img = Image.open(image_path).convert("RGB")
    img = img.resize((224, 224), Image.Resampling.LANCZOS)
    im = preprocess(img).unsqueeze(0)
    with torch.no_grad(), torch.cuda.amp.autocast():
        generated = model.generate(im)
    # Generating predicted_caption
    predicted_caption = open_clip.decode(generated[0]).split("<end_of_text>")[0].replace("<start_of_text>", "")
    class_predict = [s for s in sentences if s in predicted_caption]
    
    if ground_truth == class_predict[0]:
        predicted_correct = 1
    else:
        predicted_correct = 0
    
    pred_dict = {'img_path': [img_path], 'predicted_caption': [predicted_caption], 'ground_truth': [ground_truth], 'class_predict': [class_predict[0]], 'predicted_correct': [predicted_correct]}
    df_single = pd.DataFrame(pred_dict)
    
    return df_single


all_images = os.listdir(img_dir)
for img_path in tqdm(all_images, desc="Processing images", unit="image"):
    if '_full_' in img_path: #filaname
        continue #skip full images
    df = read_single_image(img_dir, img_path, sentences)
    df_all = pd.concat([df_all, df]).reset_index(drop=True)

# save csv file
sentences_str = "_".join(sentences)
df_all.to_csv(f"{sentences_str}.csv", index=False)
# Count the number of rows in the 'predicted_correct' column
total_rows = len(df_all)
# Count the occurrences of '1' in the 'predicted_correct' column
count_ones = df_all['predicted_correct'].sum()
print(f"Total images: {total_rows}")
print(f"predicted_correct': {count_ones}")
predicted_correct_pct = count_ones /total_rows *100
print(f"predicted_correct (%)': {predicted_correct_pct}%")